# Equity Hedging Analysis Notebook

This notebook runs analysis on the Strategy and Allocation Equity Hedge Portfolio

## Import libraries

In [1]:
import pandas as pd
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
#from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.reporting import formatter as fmt
from ipywidgets import interact, interact_manual

new_strat = False
notional_weights = []
weighted = [True, False]


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [9]:
equity_bmk = 'SPTR'
include_fi = False
strat_drop_list = ['Vortex','99%/90% Put Spread']
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## View Returns data

Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [ ]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(returns[frequency])

## Add new strategy

**Run this code if you want to add a new strategy**
* Import the daily index/returns:
    * Specify:
        * strategy_list: list of strategies 
        * filename: name of excel file
        * sheet_name: sheet name
        * data_type: index or returns data
* add it to the Equity hedge returns data

In [10]:
strategy_list = ['DSPG','3M_SMI_0.125d_0.10d']
filename = 'Credit Suisse DispHistory_SMI_0216_send.xlsx'
sheet_name = 'data'
data_type = 'index'
new_strategy = dm.get_new_strategy_returns_data(filename, sheet_name, strategy_list)
new_strategy_dict = dm.get_data_dict(new_strategy, data_type)
returns = dm.merge_dicts(returns, new_strategy_dict)
new_strat=True

In [ ]:
full_data= pd.read_excel(dm.NEW_DATA+'early_roll_vix_profit_data.xlsx',
                                           sheet_name = 'All', index_col=0)
barclays = full_data[['2W + 85% Profit']]
barclays_dict = dm.get_data_dict(barclays)
returns = dm.merge_dicts(returns,barclays_dict)
new_strat =True
weighted = [True, False]

## Get and display Notional Weights

In [4]:
#get notional weights
notional_weights = dm.get_notional_weights(returns['Monthly'])
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

notional value (Billions) for SPTR: 19
notional value (Billions) for Down Var: 1
notional value (Billions) for VOLA: 1
notional value (Billions) for Dynamic Put Spread: 1
notional value (Billions) for VRR: 1
notional value (Billions) for GW Dispersion: 1
notional value (Billions) for Corr Hedge: 0.3
notional value (Billions) for Def Var: 0.3
notional value (Billions) for DSPG: 0.5
notional value (Billions) for 3M_SMI_0.125d_0.10d: 0.5


,SPTR,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var,DSPG,3M_SMI_0.125d_0.10d
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.30,$0.30,$0.50,$0.50
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.58%,1.58%,2.63%,2.63%
Strategy Weights,0.00%,15.15%,15.15%,15.15%,15.15%,15.15%,4.55%,4.55%,7.58%,7.58%


## Compute Correlations

In [2]:
corr_freq_list = ['Daily', 'Weekly', 'Monthly']
corr_dict = summary.get_corr_data(returns, corr_freq_list, weighted, notional_weights, include_fi)

NameError: name 'returns' is not defined

## Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **weighted (True or False)** - Show Weighted Hedges or not
* **corr** - Show:
    * **corr** - full history correaltions
    * **corr_down** - risk seeking benchmark downside returns correlations or 
    * **corr_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list, weighted=weighted,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[weighted][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(returns,analytics_freq_list,weighted,notional_weights,include_fi,new_strat)

## Display Return Stats and Hedge Metrics

Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **weighted (True or False)** - Show Weighted Strats and Hedges or not
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, weighted=weighted, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[weighted]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(returns,notional_weights=notional_weights, weighted=weighted)

## Display Historical Sell Offs

Display Hisorical Sell Offs data by:
* **weighted (True or False)** - Show Weighted Hedges or not

In [ ]:
@interact
def display_selloffs(weighted=weighted):
    df_hist = hist_dict[weighted]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles

In [ ]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(returns, group = group)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list, strat=list(returns['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(returns, group = group)
    
    for col in df.columns:
        df[col] *= 100
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.update_layout(
        title = {'text':"<b>{}</b>".format(title),'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(tickfont = dict(size=12)),
        yaxis = dict(ticksuffix="%"),
        legend_title="Strategies",
        plot_bgcolor='White'
    )
    fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>SPTR Monthly Returns Ranking</b>",
        title_font = {"size": 20},showline=True,linewidth=2,linecolor='black',mirror=False)
    fig.update_yaxes(title_font_family = "Calibri",title_text = "<b></b>",title_font = {"size": 20},
        showline=True,linewidth=2,linecolor='black',mirror=False)
    fig.show()

## Compute Annual Dollar Returns

In [ ]:
annual_dollar_returns = summary.get_annual_dollar_returns(returns, notional_weights)
fmt.get_dollar_ret_styler(annual_dollar_returns)

## Display Annual Dollar Returns

In [ ]:
@interact
def display_annual_dollar_returns(strat_1=list(annual_dollar_returns.select_dtypes('float').columns)
      ,strat_2=list(annual_dollar_returns.select_dtypes('float').columns)[1:]):
    df = annual_dollar_returns.copy()
    for col in df.columns:
        if not (col == strat_1 or col ==strat_2):
            df.drop([col], axis=1, inplace=True)
    fig = px.bar(df,barmode='group')
    fig.update_layout(
        title = {'text':"<b>Annual $ Returns</b>",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(tickfont = dict(size=12)),
        legend_title="Strategies"
    )
    fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>Year</b>",
        title_font = {"size": 20})
    fig.update_yaxes(title_font_family = "Calibri",title_text = "<b>Dollars</b>",title_font = {"size": 20})
    fig.show()

## Display Monthly  Returns Table

In [7]:
month_ret_table= dm.all_strat_month_ret_table(returns["Monthly"], notional_weights = notional_weights, weighted = True)
strat_list = list(month_ret_table.keys())
@interact
def display__monthly_ret_table(strategy = strat_list):
    return fmt.get_monthly_ret_table_styler(month_ret_table[strategy])

interactive(children=(Dropdown(description='strategy', options=('SPTR', 'Down Var', 'VOLA', 'Dynamic Put Sprea…

# Create Report

Run this code below to export analysis into excel spreadsheet
* **equity_hedge_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
equity_hedge_report = 'equity_hedge_analysis_2021q2'
selloffs = True
grouped = False
rp.get_equity_hedge_report(equity_hedge_report, returns, notional_weights, include_fi, new_strat, weighted[0], selloffs, grouped)

## End